In [17]:
!pip install pygsheets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import requests
import pandas as pd
import pygsheets

In [19]:
#Mở gg sheet
client = pygsheets.authorize(service_file = '/content/refined-grammar-372209-b3fbc4b10ebf.json')

sh = client.open('API BOT Tracking')
#sh = client.open('MAIDO - DB MAGIC DATA')
wks = sh.worksheet_by_title('FB Accounts')

#Lấy dữ liệu từ CampaignPeriod và lưu thành DataFrame
df = wks.get_as_df()

#Loại bỏ những dòng có access_token là null
drop_rows = df[df['access_token']==''].index
df.drop(drop_rows,axis = 0,inplace = True)

In [20]:
def get_adaccount_id(access_token):
  
  #Tạo list rỗng để append các giá trị 
  id_lst = []
  name_lst = []
  access_token_lst = []

  #Gọi query
  query = 'https://graph.facebook.com/v15.0/me/?fields=adaccounts{name,id}&access_token={access_token}'
  query = query.replace('{access_token}',access_token)
  r = requests.get(query).json()
  accounts = r['adaccounts']['data']

  #Lặp qua các kết quả có được từ query để lưu giá trị
  for account in accounts:
    id_lst.append(account['id'])
    name_lst.append(account['name'])
    access_token_lst.append(access_token)

    print("AdAccount:",account['name'])

  #Lưu lại thành DataFrame và trả về
  df = pd.DataFrame({'account_id':id_lst,'account_name':name_lst,'access_token':access_token},columns = ['account_id','account_name','access_token'])
  return df

In [22]:
output = pd.DataFrame(columns = ['account_id','account_name','access_token'])

for i in range(0,len(df)):
  print('FB Accounts:',df.iloc[i]['name'])

  #Gọi hàm get_adaccount_id để lấy ra các thông tin adaccount cần cho việc lấy tin FB Ads
  access_token = df.iloc[i]['access_token']
  data = get_adaccount_id(access_token)
  data['app_id'] = df.iloc[i]['app_id']
  data['app_secret'] = df.iloc[i]['app_secret']

  #Lưu vào DataFrame output
  output = pd.concat([output,data], axis =0)
  print('----------------')
  print('')

#Ghi vào gg sheet
write_sheet = sh.worksheet_by_title('AdAccounts')
write_sheet.clear()
write_sheet.set_dataframe(output,start = "A1",copy_head = True)

FB Accounts: HT
AdAccount: Maido Agency Co., LTD
AdAccount: Trang Ho
AdAccount: Công Ty TNHH XNK Á Âu
----------------

FB Accounts: KN
AdAccount: Maido Agency
AdAccount: The Ruong
AdAccount: Khoi Nguyen
----------------

FB Accounts: TN
AdAccount: Tan Nguyen
AdAccount: Công Ty TNHH Quảng Cáo Maido
----------------

